In [ ]:
import pandas as pd
import numpy as np
!pip install fuzzywuzzy
from fuzzywuzzy import process, fuzz


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def changing_set_creation(high_score_sort):
    new_df = high_score_sort[high_score_sort['score_sort'] >= 82]
    return new_df

In [ ]:
def changing_set_unit_update(changing_set, df):
  for index, row in changing_set.iterrows():
     df['unit'] = df['unit'].replace(changing_set.loc[index, 'match_sort'], changing_set.loc[index, 'unit_sort'])
  return df

def changing_set_address_update(changing_set, newdf):
    for index, row in changing_set.iterrows():
      newdf['addres'] = newdf['addres'].replace(changing_set.loc[index, 'match_sort'], changing_set.loc[index, 'unit_address'])
    return newdf

def changing_set_loc1_update(changing_set, newdf):
    for index, row in changing_set.iterrows():
      newdf['location_1'] = newdf['location_1'].replace(changing_set.loc[index, 'match_sort'], changing_set.loc[index, 'location1_sort'])
    return newdf

def changing_set_loc2_update(changing_set, newdf):
    for index, row in changing_set.iterrows():
      newdf['location_2'] = newdf['location_2'].replace(changing_set.loc[index, 'match_sort'], changing_set.loc[index, 'location2_sort'])
    return newdf

In [ ]:
df = pd.read_csv("Dubai_data_after_merge.csv")
df.head()

,url,bedrooms,bathrooms,city,location_1,location_2,addres,price,size,price_per_unit,type,unit,rent_or_sale,web_name,source,lat,lon
0,https://www.propertyfinder.ae/en/plp/buy/townh...,4,4,Dubai,Tilal Al Ghaf,Tilal Al Ghaf,"Aura, Tilal Al Ghaf, Dubai",4000000,229,17467.25,residential,Townhouse,sale,property finder,scraped,25.023709,55.223482
1,https://www.propertyfinder.ae/en/plp/buy/apart...,1,1,Dubai,Jumeirah,La Mer,"Port de La Mer, La Mer, Jumeirah, Dubai",2200000,76,28947.37,residential,Apartment,sale,property finder,scraped,25.206960,55.247497
2,https://www.propertyfinder.ae/en/plp/buy/apart...,2,2,Dubai,Jumeirah,La Mer,"Le Pont, La Mer, Jumeirah, Dubai",5000000,113,44247.79,residential,Apartment,sale,property finder,scraped,25.206960,55.247497
3,https://www.propertyfinder.ae/en/plp/buy/apart...,2,2,Dubai,Jumeirah Village Circle,Jumeirah Village Circle,"Maimoon Gardens by Fakhruddin Properties, Jume...",1510000,90,16777.78,residential,Apartment,sale,property finder,scraped,25.071164,55.210889
4,https://www.propertyfinder.ae/en/plp/buy/villa...,4,4,Dubai,Dubai Hills Estate,Maple at Dubai Hills Estate,"Maple 3, Maple at Dubai Hills Estate, Dubai Hi...",4250000,230,18478.26,residential,Villa,sale,property finder,scraped,25.122615,55.275120


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313068 entries, 0 to 313067
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   url             313068 non-null  object 
 1   bedrooms        313068 non-null  int64  
 2   bathrooms       313068 non-null  int64  
 3   city            313068 non-null  object 
 4   location_1      313068 non-null  object 
 5   location_2      313068 non-null  object 
 6   addres          313068 non-null  object 
 7   price           313068 non-null  int64  
 8   size            313068 non-null  int64  
 9   price_per_unit  313068 non-null  float64
 10  type            313068 non-null  object 
 11  unit            313068 non-null  object 
 12  rent_or_sale    313068 non-null  object 
 13  web_name        313068 non-null  object 
 14  source          313068 non-null  object 
 15  lat             313068 non-null  float64
 16  lon             313068 non-null  float64
dtypes: float64

In [ ]:
#remove leading and trailing spaces (if available) in every column, and then print out their number of unique values.
from numpy.lib.utils import source
for col in df[['url','city','location_1','location_2','addres','type','unit','rent_or_sale','web_name','source']]:
    df[col] = df[col].str.strip()
    print('Number of unique values in ' + str(col) +': ' + str(df[col].nunique()))

Number of unique values in url: 263688
Number of unique values in city: 1
Number of unique values in location_1: 157
Number of unique values in location_2: 2183
Number of unique values in addres: 7605
Number of unique values in type: 2
Number of unique values in unit: 60
Number of unique values in rent_or_sale: 2
Number of unique values in web_name: 3
Number of unique values in source: 1


In [ ]:
unique_address = df['addres'].unique().tolist()
sorted(unique_address)[:20]
unique_location1 = df['location_1'].unique().tolist()
sorted(unique_address)[:20]
unique_location2 = df['location_2'].unique().tolist()
sorted(unique_address)[:20]
unique_unit = df['unit'].unique().tolist()
sorted(unique_unit)[:20]

['Apartment',
 'Bulk Rent Unit',
 'Bulk Sale Unit',
 'Bulk Unit',
 'Bungalow',
 'Business Centre',
 'Co-working space',
 'Commercial Building',
 'Commercial Floor',
 'Commercial Plot',
 'Commercial Villa',
 'Compound',
 'Duplex',
 'Factory',
 'Farm',
 'Full Floor',
 'Half Floor',
 'Hotel & Hotel Apartment',
 'Hotel Apartment',
 'Industrial Land']

In [ ]:
score_sort = [(x,) + i
             for x in unique_address
             for i in process.extract(x, unique_address,scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['unit_address','match_sort','score_sort'])
similarity_sort['sorted_address_sort']= np.minimum(similarity_sort['unit_address'],similarity_sort['match_sort'])
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['unit_address'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_address_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_address_sort',axis=1).copy()
high_score_sort.groupby(['unit_address','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
unit_address,score_sort,
"Expo Golf Villas, EMAAR South, Dubai South, Dubai",100,"Expo Golf Villas, Emaar South, Dubai South, Dubai"
"BELAIR The Trump Estates, Trump Estates, DAMAC Hills, Dubai",100,"BelAir The Trump Estates, Trump Estates, DAMAC..."
"LIV Lux, Dubai Marina, Dubai",100,"Liv Lux, Dubai Marina, Dubai"
"Beach Isle, EMAAR Beachfront, Dubai Harbour, Dubai",100,"Beach Isle, Emaar Beachfront, Dubai Harbour, D..."
"Park Point Building B, Park Point, Dubai Hills Estate, Dubai",100,"Park Point building B, Park Point, Dubai Hills..."
...,...,...
"Wasl Aqua, Al Karama, Dubai",80,"Wasl R421, Al Karama, Dubai"
"Marinascape Marina Homes, Marinascape, Dubai Marina, Dubai",80,"Marinascape Oceanic, Marinascape, Dubai Marina..."
"Bonaire Tower, Park Island, Dubai Marina, Dubai",80,"Fairfield Tower, Park Island, Dubai Marina, Dubai"


In [ ]:
changing_set = changing_set_creation(high_score_sort)
df = changing_set_address_update(changing_set,df)
df['addres'].nunique()

3280

In [ ]:
score_sort = [(x,) + i
             for x in unique_location1
             for i in process.extract(x, unique_location1,     scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['location1_sort','match_sort','score_sort'])
similarity_sort['sorted_loc1_sort']= np.minimum(similarity_sort['location1_sort'],similarity_sort['match_sort'])
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['location1_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_loc1_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_loc1_sort',axis=1).copy()
high_score_sort.groupby(['location1_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
location1_sort,score_sort,
Nad Al Sheba,96,Nadd Al Sheba
Dubai Land,95,Dubailand
Dubai Healthcare City,95,Dubai Healthcare City 2
Arabian Ranches,94,"Arabian Ranches 3, Arabian Ranches 2"
Arabian Ranches 2,94,Arabian Ranches 3
World Trade Center,94,World Trade Centre
Wadi Al Safa 2,93,Wadi Al Safa 3
Emirates Hills,93,Emirates Hills 2
Jumeirah Beach Residence,92,Jumeirah Beach Residence (JBR)


In [ ]:
changing_set = changing_set_creation(high_score_sort)
df = changing_set_loc1_update(changing_set,df)
df['location_1'].nunique()

138

In [ ]:
score_sort = [(x,) + i
             for x in unique_location2
             for i in process.extract(x, unique_location2,     scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['location2_sort','match_sort','score_sort'])
similarity_sort.head(10)
similarity_sort['sorted_loc2_sort']= np.minimum(similarity_sort['location2_sort'],similarity_sort['match_sort'])
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['location2_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_loc2_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_loc2_sort',axis=1).copy()
high_score_sort.groupby(['location2_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
location2_sort,score_sort,
Zen By Indigo,100,Zen by Indigo
DAMAC Maison Canal Views,100,Damac Maison Canal Views
Green Community (Motor City),100,Green Community Motor City
Kingdom Of Sheba,100,Kingdom of Sheba
The LOFT Offices,100,The Loft Offices
...,...,...
Mina Rashid,80,Mina Road
Skyline Residence,80,Sunshine Residence
Al Twar 3,80,Al Warqaa 3


In [ ]:
changing_set = changing_set_creation(high_score_sort)
newdf = changing_set_loc2_update(changing_set, df)
df['location_2'].nunique()

1641

In [ ]:
score_sort = [(x,) + i
             for x in unique_unit
             for i in process.extract(x, unique_unit,     scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['unit_sort','match_sort','score_sort'])
similarity_sort['sorted_unit_sort']= np.minimum(similarity_sort['unit_sort'],similarity_sort['match_sort'])
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['unit_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_unit_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_unit_sort',axis=1).copy()
high_score_sort.groupby(['unit_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
unit_sort,score_sort,
Showroom,100,showroom
Residential Building,100,residential-building
Commercial Building,100,commercial-building
Warehouse,100,warehouse
Shop,100,shop
Commercial Floor,100,commercial-floor
Retail,100,retail
Commercial Villa,100,commercial-villa
Factory,100,factory


In [ ]:
changing_set = changing_set_creation(high_score_sort)
df = changing_set_unit_update(changing_set, df)
df['unit'].nunique()

40

In [ ]:
for col in df[['url','city','location_1','location_2','addres','type','unit','rent_or_sale','web_name','source']]:
    df[col] = df[col].str.strip()
    print('Number of unique values in ' + str(col) +': ' + str(df[col].nunique()))

Number of unique values in url: 263688
Number of unique values in city: 1
Number of unique values in location_1: 138
Number of unique values in location_2: 1641
Number of unique values in addres: 3280
Number of unique values in type: 2
Number of unique values in unit: 40
Number of unique values in rent_or_sale: 2
Number of unique values in web_name: 3
Number of unique values in source: 1


In [ ]:
df.to_csv("Dubai_Data_after_merge&clean.csv")